In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee as mc

%matplotlib inline

In [27]:
def createMockCluster(rin,rout,nmean,k,smax,m,vmean,threeD=False):
    ntot = np.random.poisson(lam=nmean)
    
    nu0 = ntot/(4*np.pi*(np.log(rout/rin))) if k==3. else ntot*(3.-k)/(4*np.pi*(rout**(3.-k)-rin**(3.-k)))
    #rho = 4*np.pi*nu0*np.random.uniform(low=rin,high=rout,size=ntot)**(2.-k)
    rho = np.power(np.random.uniform(4*np.pi*nu0*rin**(2.-k),4*np.pi*nu0*rout**(2.-k),size=ntot)/(4*np.pi*nu0),1./(2-k))
    phi = np.random.uniform(0.,2*np.pi,ntot)
    theta = np.arccos(np.random.uniform(-1,1,ntot))
    
    x = rho*np.sin(theta)*np.cos(phi)
    y = rho*np.sin(theta)*np.sin(phi)
    
    vobs = np.array([np.random.normal(loc=vmean,scale=np.sqrt(3)*smax/(i**m)) for i in rho])
    
    if threeD:
        z = rho*np.cos(theta)
        return pd.DataFrame(np.hstack((x.reshape((ntot,1)),y.reshape((ntot,1)),z.reshape((ntot,1)),\
                                      vobs.reshape((ntot,1)),rho.reshape((ntot,1)))),columns=['x','y','z','vobs','rho'])
    else:
        r = np.sqrt(np.power(x,2.)+np.power(y,2.))
        return pd.DataFrame(np.hstack((r.reshape((ntot,1)),vobs.reshape((ntot,1)))),columns=['r','vobs'])

In [28]:
#load data

data=pd.read_csv('continuousMock0.csv')

def defineAnnuli(r,nAnn):
    return np.append(0.,np.sort(r)[np.linspace(0,len(r)-1,nAnn).astype(int)][1:])

nAnn = 10
annuli = defineAnnuli(data['r'],nAnn)

In [29]:
def labelGalaxies(data,annuli):
    #adds a column to data indicating the galaxy's membership in an annulus
    bla = np.zeros(len(data['r']))
    for i in range(len(annuli)-1):
        bla += i*np.array(np.logical_and(data['r']>annuli[i],data['r']<=annuli[i+1]).astype(int))
    return bla

def galInAnn(data,annuli):
    #calculates the number of galaxies in all the annuli
    return np.array([(data['annuli']==i).sum() for i in range(len(annuli)-1)])

def sigInAnn(data,annuli):
    return [np.std(data['vobs'][data['annuli']==i],ddof=1) for i in range(len(annuli)-1)]

def getProfiles(annuli,rin,rout,nmean,k,smax,m,vmean):
    mock = createMockCluster(rin,rout,nmean,k,smax,m,vmean)
    mock['annuli'] = labelGalaxies(mock,annuli)
    return galInAnn(mock,annuli),sigInAnn(mock,annuli)

In [31]:
data['annuli'] = labelGalaxies(data,annuli)

nObs = galInAnn(data,annuli)
sObs = sigInAnn(data,annuli)

print nObs,sObs

[113 112 112 112 113 112 112 112 113] [1533.5686970592865, 1708.8471911765737, 1672.5203324606516, 1657.0006896288683, 1365.0966816840385, 1551.392522463407, 1520.69833183794, 1134.8559673428142, 849.5847196305533]


In [22]:
#all log likelihoods

#make a table of log(n!)-s for n=1,...
#factorialTable = np.cumsum(np.append(0,[np.log(i) for i in range(1,len(data['r']))]))

def logPriorK(k):
    #starting gaussian
    #nu0 = 3. #mean prior density index
    #sigma = 1.
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorM(m):
    #starting gaussian
    #nu0 = -.2 #mean prior sigma index
    #sigma = 0.1
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorSigma0(sigma0):
    #starting gaussian
    #nu0 = 1000 #mean prior sigma index
    #sigma = 500
    return 0.0
    #return -0.5*np.log(2*np.pi*sigma**2)-0.5*(k-nu0)**2/(2*sigma**2)

def logPriorNmean(nmean):
    return 0.0

def logPriorNtot(ntot):
    return 0.0

def logLikelihoodNu(prediction,data):
    return np.sum(data*np.log(prediction)-prediction)

def logLikelihoodSigma(vObs,vPred,nPred):
    # distribution of variances is chisq, as shown, for example, at
    # https://onlinecourses.science.psu.edu/stat414/node/174
    return np.sum(np.log(stats.chi2.pdf((nPred-1.)*np.power(vObs,2.)/np.power(vPred,2.),nPred-1)))

def logPosterior(nmean,k,smax,m,annuli,rin,rout,nObs,sObs):
    nPred,sPred = getProfiles(annuli,rin,rout,1000*nmean,k,smax,m,0)
    nPred/=nmean
    return logPriorK(k)+logPriorM(m)+logPriorSigma0(smax)+logPriorNmean(nmean)+\
           logLikelihoodNu(nPred,nObs)+logLikelihoodSigma(sObs,sPred,nPred)

In [23]:
ntot=1000
bla1,bla2 = getProfiles(annuli,1,1000,1000*ntot,3,1000,0.2,0)
bla1/=ntot
bla2/=np.sqrt(ntot)

print bla1
print bla2

[ 99 116 106 111 124 107 104 110 118]
[ 51.44193012  52.21131893  52.15547032  50.26068846  48.34638222
  45.89147195  43.19688007  39.07649967  30.82787527]


In [ ]:
nwalkers = 10
ndim = 4


In [ ]:
sampler = mc.EnsembleSampler(nwalkers, ndim, logPosterior, args=[annuli,rin,rout,nObs,sObs])